In [2]:
!pip install scipy==1.10.1 gensim nltk pyldavis


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
from gensim import models
import glob
import nltk
from nltk.stem import *
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

In [4]:
nltk.download('stopwords')
stopwords = set(nltk.corpus.stopwords.words('english'))

nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rmartins/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/rmartins/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
custom_stopwords = ['one', 'two', 'also', 'c', 'however']

aux_texts = []
filenames = []

# The two batches are slightly different
for filename in glob.glob('txts-webofscience-643-2177/*.txt'):
    filenames.append(filename)
    with open(filename, encoding='utf16') as file:
        # Ignore first line
        text = file.readline()
        text = file.readline().split('\t')[4].strip()
        aux_texts.append(text)

for filename in glob.glob('txts-webofscience-*/*.txt'):
    filenames.append(filename)
    with open(filename, encoding='utf16') as file:
        text = file.readline().split('\t')[4].strip()
        aux_texts.append(text)

stemmer = PorterStemmer()

stopwords.update(custom_stopwords)

texts = []

for text in aux_texts:
    words = set(word.lower() for word in nltk.word_tokenize(text) if word.isalpha())
    words = [word for word in words if word not in stopwords]
    #words = [stemmer.stem(word) for word in words if word not in stopwords]        

    # This is arbitrary; trying to remove noise and errors
    if len(words) > 10:
        texts.append(words)
        #print(words)

In [15]:
#filenames

In [11]:
our_dict = Dictionary(texts)
our_corpus = [our_dict.doc2bow(text) for text in texts]

In [17]:
%%time

NUM_TOPICS=10

# Train the model on the corpus.
our_lda_model = models.ldamodel.LdaModel(our_corpus, num_topics=NUM_TOPICS)

CPU times: user 3.69 s, sys: 3.77 ms, total: 3.69 s
Wall time: 3.69 s


In [30]:
vis_data = gensimvis.prepare(our_lda_model, our_corpus, our_dict)
pyLDAvis.display(vis_data)

In [22]:
# prepare the list of documents per topic
docs_per_topic = {i:[] for i in range(NUM_TOPICS)}

for i, bow in enumerate(our_corpus):
    t = max(our_lda_model.get_document_topics(bow), key=lambda x: x[1])
    docs_per_topic[t[0]].append(filenames[i])

In [26]:
[len(x) for _, x in docs_per_topic.items()]

[136, 90, 259, 92, 103, 125, 168, 249, 205, 266]

In [29]:
for i, docs in docs_per_topic.items():
    print(f'Topic #{i}:')
    for d in docs:
        print('   ', d)

Topic #0:
    txts-webofscience-643-2177/806-806.txt
    txts-webofscience-643-2177/337-337.txt
    txts-webofscience-643-2177/157-157.txt
    txts-webofscience-643-2177/69-69.txt
    txts-webofscience-643-2177/671-671.txt
    txts-webofscience-643-2177/841-841.txt
    txts-webofscience-643-2177/537-537.txt
    txts-webofscience-643-2177/656-656.txt
    txts-webofscience-643-2177/1534-1534.txt
    txts-webofscience-643-2177/1421-1421.txt
    txts-webofscience-643-2177/921-921.txt
    txts-webofscience-643-2177/1133-1133.txt
    txts-webofscience-643-2177/1264-1264.txt
    txts-webofscience-643-2177/28-28.txt
    txts-webofscience-643-2177/773-773.txt
    txts-webofscience-643-2177/115-115.txt
    txts-webofscience-643-2177/1167-1167.txt
    txts-webofscience-643-2177/1042-1042.txt
    txts-webofscience-643-2177/952-952.txt
    txts-webofscience-643-2177/1294-1294.txt
    txts-webofscience-643-2177/1156-1156.txt
    txts-webofscience-643-2177/426-426.txt
    txts-webofscience-643-2177/1